# 🛒 Amazon Scraping - Complete Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vzucher/brightdata-sdk-python/blob/master/notebooks/03_amazon_scraping.ipynb)

Master Amazon data scraping: products, reviews, sellers, and competitive analysis.

## What You'll Learn
1. Scraping product details
2. Extracting reviews
3. Seller information
4. Price tracking
5. Competitive analysis

---


In [ ]:
%pip install brightdata-sdk pandas matplotlib -q

import pandas as pd
import matplotlib.pyplot as plt
from brightdata import BrightDataClient
from brightdata.payloads import AmazonProductPayload, AmazonReviewPayload

API_TOKEN = "your_api_token_here"
client = BrightDataClient(token=API_TOKEN)
print("✅ Ready!")


## 📦 1. Scrape Product Details


In [ ]:
# Scrape a product with validation
payload = AmazonProductPayload(
    url="https://www.amazon.com/dp/B0CRMZHDG8",
    reviews_count=50,  # Get up to 50 reviews
    images_count=10    # Get up to 10 images
)

print(f"ASIN: {payload.asin}")
print(f"Domain: {payload.domain}")
print(f"Secure: {payload.is_secure}")

result = client.scrape.amazon.products(**payload.to_dict())

if result.success:
    print(f"\n✅ Success!")
    print(f"Title: {result.data.get('title')}")
    print(f"Price: {result.data.get('final_price')}")
    print(f"Rating: {result.data.get('rating')}")
    print(f"Reviews: {result.data.get('reviews_count')}")
    print(f"Availability: {result.data.get('availability')}")
    print(f"\nCost: ${result.cost:.4f}")
else:
    print(f"❌ Failed: {result.error}")


## ⭐ 2. Scrape Product Reviews


In [ ]:
# Get reviews from last 30 days
reviews_result = client.scrape.amazon.reviews(
    url="https://www.amazon.com/dp/B0CRMZHDG8",
    pastDays=30
)

if reviews_result.success and reviews_result.data:
    reviews_df = pd.DataFrame(reviews_result.data.get('reviews', []))
    print(f"✅ Got {len(reviews_df)} reviews")
    print(f"\nSample review:")
    if len(reviews_df) > 0:
        display(reviews_df[['rating', 'title', 'body']].head(3))


## 💰 3. Price Comparison Analysis

Scrape multiple similar products and compare prices:


In [ ]:
# Compare competing products
competitor_asins = ["B0CRMZHDG8", "B09B9C8K3T", "B0CX23V2ZK"]
products = []

for asin in competitor_asins:
    url = f"https://www.amazon.com/dp/{asin}"
    result = client.scrape.amazon.products(url=url)
    
    if result.success:
        products.append({
            'asin': asin,
            'title': result.data.get('title', 'N/A')[:50],
            'price': result.data.get('final_price'),
            'rating': result.data.get('rating'),
            'reviews': result.data.get('reviews_count'),
        })

df = pd.DataFrame(products)
print("📊 Price Comparison:")
display(df)

# Find best value
if len(df) > 0:
    print(f"\n💎 Best Rating: {df.loc[df['rating'].idxmax(), 'title']}")
    print(f"🔥 Most Reviews: {df.loc[df['reviews'].idxmax(), 'title']}")


## 📊 4. Visualization


In [ ]:
# Create comparison chart
if len(df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Price comparison
    axes[0].bar(range(len(df)), df['price'].str.replace('$','').astype(float))
    axes[0].set_title('Price Comparison', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Price ($)')
    axes[0].set_xticks(range(len(df)))
    axes[0].set_xticklabels([f"ASIN {i+1}" for i in range(len(df))])
    
    # Rating comparison
    axes[1].bar(range(len(df)), df['rating'], color='green')
    axes[1].set_title('Rating Comparison', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Rating (stars)')
    axes[1].set_xticks(range(len(df)))
    axes[1].set_xticklabels([f"ASIN {i+1}" for i in range(len(df))])
    axes[1].set_ylim([0, 5])
    
    plt.tight_layout()
    plt.show()


## ✅ Summary

You learned:
- ✅ Scraping Amazon products with validation
- ✅ Extracting product reviews
- ✅ Price comparison analysis
- ✅ Data visualization

## 🎓 Next: [LinkedIn Jobs Analysis](./04_linkedin_jobs.ipynb)

**Happy Amazon Scraping! 🛒**
